<a href="https://colab.research.google.com/github/Ch256981/IMLP347/blob/main/HW2_Ko_I_Hung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BUDT 758J - HW2

## I-Hung Ko

In [1]:
# install PySpark
!pip install -q pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
# create a SparkSession from pyspark.sql, which is the entry point to Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('HW2').getOrCreate()

In [3]:
# verify
spark

# Load Dataset

In [4]:
# 703
spark.read.json('hw1_customer.json').createOrReplaceTempView('Customer')
spark.read.json('hw1_order.json').createOrReplaceTempView('Order')
spark.read.json('hw1_orderline.json').createOrReplaceTempView('OrderLine')
spark.read.json('hw1_product.json').createOrReplaceTempView('Product')

In [5]:
# 704
import pandas as pd
pd_customer = pd.read_json('hw1_customer.json')
pd_order = pd.read_json('hw1_order.json')
pd_orderline = pd.read_json('hw1_orderline.json')
pd_product = pd.read_json('hw1_product.json')

In [6]:
# 758J
sp_customer = spark.read.json('hw1_customer.json')
sp_order = spark.read.json('hw1_order.json')
sp_orderline = spark.read.json('hw1_orderline.json')
sp_product = spark.read.json('hw1_product.json')

# Data Manipulation

#5. How many products (i.e., distinct count) that were ordered (i.e., in each product line) and what was the total ordered quantity in each product line (in the ascending order of product line id)?

In [61]:
# 703
spark.sql('''SELECT p.productLineId,
           COUNT(DISTINCT l.productId) AS `Count of Products Ordered`,
           SUM(l.orderedQuantity) AS `Total Ordered Quantity`
       FROM Orderline l LEFT JOIN Product p ON l.productId = p.productId
       GROUP BY p.productLineId
       ORDER BY p.productLineId ASC''').show()

+-------------+-------------------------+----------------------+
|productLineId|Count of Products Ordered|Total Ordered Quantity|
+-------------+-------------------------+----------------------+
|            1|                        5|                    29|
|            2|                        1|                     2|
|            3|                        2|                    20|
+-------------+-------------------------+----------------------+



In [54]:
# 704
pd.merge(pd_orderline, pd_product, how='left', on='productId')\
 .groupby('productLineId', as_index=False)\
 .agg({'productId' : 'nunique', 'orderedQuantity': 'sum'})\
 .rename(columns={'productId': 'Count of Products Ordered', 'orderedQuantity': 'Total Ordered Quantity'})

,productLineId,Count of Products Ordered,Total Ordered Quantity
0,1,5,29
1,2,1,2
2,3,2,20


In [80]:
# 758J
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.countDistinct.html
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.sum.html
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import sum
sp_orderline.join(sp_product, 'productId', how='left')\
      .groupBy('productLineId')\
      .agg(countDistinct('productId')\
        .alias('Count of Products Ordered'),
         sum('orderedQuantity')\
         .alias('Total Ordered Quantity'))\
      .orderBy('productLineId').show()

+-------------+-------------------------+----------------------+
|productLineId|Count of Products Ordered|Total Ordered Quantity|
+-------------+-------------------------+----------------------+
|            1|                        5|                    29|
|            2|                        1|                     2|
|            3|                        2|                    20|
+-------------+-------------------------+----------------------+



#6. What are the id, name (no truncation), finish, and revenue (0 if the product was never been ordered) for all products (in the ascending order of product id)?

In [62]:
# 703
from os import truncate
spark.sql('''SELECT p.productId,
           p.productDescription,
           p.productFinish,
           COALESCE (SUM(p.productPrice * l.orderedQuantity), 0) AS `Revenue`
       FROM Product p LEFT JOIN Orderline l ON p.productId = l.productId
       GROUP BY p.productId, p.productDescription, p.productFinish
       ORDER BY p.productId ASC''').show(truncate=False)

+---------+-----------------------+-------------+-------+
|productId|productDescription     |productFinish|Revenue|
+---------+-----------------------+-------------+-------+
|1        |Cherry End Table       |Cherry       |875    |
|2        |Birch Coffee Tables    |Birch        |800    |
|3        |Oak Computer Desk      |Oak          |8250   |
|4        |Entertainment Center   |Cherry       |11550  |
|5        |Writer's Desk          |Oak          |650    |
|6        |8-Drawer Dresser       |Birch        |1500   |
|7        |48 Bookcase            |Walnut       |750    |
|8        |48 Bookcase            |Oak          |2625   |
|9        |96 Bookcase            |Walnut       |0      |
|10       |96 Bookcase            |Oak          |0      |
|11       |4-Drawer Dresser       |Oak          |0      |
|12       |8-Drawer Dresser       |Oak          |0      |
|13       |Nightstand             |Cherry       |0      |
|14       |Writer's Desk          |Birch        |0      |
|17       |Hig

In [58]:
# 704
pd.merged_df = pd.merge(pd_product[['productId', 'productDescription',\
                   'productFinish', 'productPrice']],\
             pd_orderline, how='left', on='productId')
pd.merged_df['Revenue'] = pd.merged_df['productPrice'] * pd.merged_df['orderedQuantity']
pd.merged_df.groupby(['productId', 'productDescription', 'productFinish'], as_index=False)\
      .agg({'Revenue': 'sum'})\
      .fillna({'Revenue': 0})\
      .astype({'Revenue': int})

,productId,productDescription,productFinish,Revenue
0,1,Cherry End Table,Cherry,875
1,2,Birch Coffee Tables,Birch,800
2,3,Oak Computer Desk,Oak,8250
3,4,Entertainment Center,Cherry,11550
4,5,Writer's Desk,Oak,650
5,6,8-Drawer Dresser,Birch,1500
6,7,48 Bookcase,Walnut,750
7,8,48 Bookcase,Oak,2625
8,9,96 Bookcase,Walnut,0
9,10,96 Bookcase,Oak,0


In [81]:
# 758J
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.coalesce.html
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.expr.html
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.sum.html
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.lit.html
from pyspark.sql.functions import coalesce
from pyspark.sql.functions import expr
from pyspark.sql.functions import sum
from pyspark.sql.functions import lit
sp_product.join(sp_orderline, 'productId', how='left')\
     .groupBy('productId', 'productDescription', 'productFinish')\
     .agg(coalesce(expr('sum(productPrice * orderedQuantity)'), lit(0))\
       .alias('Revenue'))\
     .orderBy('productId').show(truncate=False)

+---------+-----------------------+-------------+-------+
|productId|productDescription     |productFinish|Revenue|
+---------+-----------------------+-------------+-------+
|1        |Cherry End Table       |Cherry       |875    |
|2        |Birch Coffee Tables    |Birch        |800    |
|3        |Oak Computer Desk      |Oak          |8250   |
|4        |Entertainment Center   |Cherry       |11550  |
|5        |Writer's Desk          |Oak          |650    |
|6        |8-Drawer Dresser       |Birch        |1500   |
|7        |48 Bookcase            |Walnut       |750    |
|8        |48 Bookcase            |Oak          |2625   |
|9        |96 Bookcase            |Walnut       |0      |
|10       |96 Bookcase            |Oak          |0      |
|11       |4-Drawer Dresser       |Oak          |0      |
|12       |8-Drawer Dresser       |Oak          |0      |
|13       |Nightstand             |Cherry       |0      |
|14       |Writer's Desk          |Birch        |0      |
|17       |Hig

#7. What are the id, date, customer name, and overall price for each order (in the ascending order of order id)?

In [63]:
# 703
spark.sql('''SELECT o.orderId,
           o.orderDate,
           c.customerName,
           SUM(p.productPrice * l.orderedQuantity) AS `Overall Price`
       FROM Order o LEFT JOIN Customer c ON o.customerId = c.customerId
       JOIN Orderline l ON o.orderId = l.orderId
       JOIN Product p ON l.productId = p.productId
       GROUP BY o.orderId, o.orderDate, c.customerName
       ORDER BY o.orderId ASC''').show()

+-------+----------+--------------------+-------------+
|orderId| orderDate|        customerName|Overall Price|
+-------+----------+--------------------+-------------+
|   1001|2010-10-21|Contemporary Casuals|         2400|
|   1002|2010-10-21| California Classics|         3750|
|   1003|2010-10-22|     Mountain Scenes|         2250|
|   1004|2010-10-22|         Impressions|         1850|
|   1005|2010-10-24|    Home Furnishings|         4950|
|   1006|2010-10-24|     Value Furniture|         2600|
|   1007|2010-10-27|American Euro Lif...|          925|
|   1008|2010-10-30|Battle Creek Furn...|         2775|
|   1009|2010-11-05|   Eastern Furniture|         3750|
|   1010|2010-11-05|Contemporary Casuals|         1750|
+-------+----------+--------------------+-------------+



In [64]:
# 704
pd.merged_df = pd.merge(
  pd.merge(
    pd.merge(
      pd_order, pd_customer,
      how='left', on='customerId'
    ),
    pd_orderline,
    how='left', on='orderId'
  ),
  pd_product,
  how='left', on='productId'
)
pd.merged_df['Overall Price'] = pd.merged_df['productPrice'] * pd.merged_df['orderedQuantity']
pd.merged_df.groupby(['orderId', 'orderDate', 'customerName'], as_index=False)\
      .agg({'Overall Price': 'sum'})\
      .sort_values('orderId')

,orderId,orderDate,customerName,Overall Price
0,1001,2010-10-21,Contemporary Casuals,2400
1,1002,2010-10-21,California Classics,3750
2,1003,2010-10-22,Mountain Scenes,2250
3,1004,2010-10-22,Impressions,1850
4,1005,2010-10-24,Home Furnishings,4950
5,1006,2010-10-24,Value Furniture,2600
6,1007,2010-10-27,American Euro Lifestyles,925
7,1008,2010-10-30,Battle Creek Furniture,2775
8,1009,2010-11-05,Eastern Furniture,3750
9,1010,2010-11-05,Contemporary Casuals,1750


In [82]:
# 758J
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.sum.html
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.expr.html
from pyspark.sql.functions import sum
from pyspark.sql.functions import expr
sp_order.join(sp_customer, 'customerId', 'left')\
    .join(sp_orderline, 'orderId', 'left')\
    .join(sp_product, 'productId', 'left')\
    .groupBy('orderId', 'orderDate', 'customerName')\
    .agg(sum(expr('productPrice * orderedQuantity'))\
      .alias('Overall Price'))\
    .orderBy('orderId').show()

+-------+----------+--------------------+-------------+
|orderId| orderDate|        customerName|Overall Price|
+-------+----------+--------------------+-------------+
|   1001|2010-10-21|Contemporary Casuals|         2400|
|   1002|2010-10-21| California Classics|         3750|
|   1003|2010-10-22|     Mountain Scenes|         2250|
|   1004|2010-10-22|         Impressions|         1850|
|   1005|2010-10-24|    Home Furnishings|         4950|
|   1006|2010-10-24|     Value Furniture|         2600|
|   1007|2010-10-27|American Euro Lif...|          925|
|   1008|2010-10-30|Battle Creek Furn...|         2775|
|   1009|2010-11-05|   Eastern Furniture|         3750|
|   1010|2010-11-05|Contemporary Casuals|         1750|
+-------+----------+--------------------+-------------+



#8. What are the id, name (no truncation), address (full address in one column), and count of orders (0 if never ordered) for each customer (in the ascending order of customer id)?

In [69]:
# 703
spark.sql('''SELECT c.customerId,
           c.customerName,
           CONCAT(c.customerStreet, ', ',c.customerCity, ', ',c.customerState, ', ',c.customerZip) AS `Address`,
           COALESCE(COUNT(o.orderId), 0) AS `Count of Orders`
       FROM Customer c LEFT JOIN Order o ON c.customerId = o.customerId
       GROUP BY c.customerId, c.customerName, c.customerStreet, c.customerCity, c.customerState, c.customerZip
       ORDER BY c.customerId ASC''').show(truncate=False)

+----------+------------------------+---------------------------------------------------+---------------+
|customerId|customerName            |Address                                            |Count of Orders|
+----------+------------------------+---------------------------------------------------+---------------+
|1         |Contemporary Casuals    |1355 S Hines Blvd, Gainesville, FL, 32601-2871     |2              |
|2         |Value Furniture         |15145 S.W. 17th St., Plano, TX, 75094-7743         |1              |
|3         |Home Furnishings        |1900 Allard Ave., Albany, NY, 12209-1125           |1              |
|4         |Eastern Furniture       |1925 Beltline Rd., Carteret, NJ, 07008-3188        |1              |
|5         |Impressions             |5585 Westcott Ct., Sacramento, CA, 94206-4056      |1              |
|6         |Furniture Gallery       |325 Flatiron Dr., Boulder, CO, 80514-4432          |0              |
|7         |Period Furniture        |394 Rainb

In [71]:
# 704
pd_customer['Address'] = (
    pd_customer['customerStreet'] + ', ' +
    pd_customer['customerCity'] + ', ' +
    pd_customer['customerState'] + ', ' +
    pd_customer['customerZip']
)
pd.merge(pd_customer, pd_order, how='left', on='customerId')\
 .groupby(['customerId', 'customerName', 'Address'], as_index=False)\
 .agg({'orderId': 'count'})\
 .rename(columns={'orderId': 'Count of Orders'})\
 .sort_values('customerId')

,customerId,customerName,Address,Count of Orders
0,1,Contemporary Casuals,"1355 S Hines Blvd, Gainesville, FL, 32601-2871",2
1,2,Value Furniture,"15145 S.W. 17th St., Plano, TX, 75094-7743",1
2,3,Home Furnishings,"1900 Allard Ave., Albany, NY, 12209-1125",1
3,4,Eastern Furniture,"1925 Beltline Rd., Carteret, NJ, 07008-3188",1
4,5,Impressions,"5585 Westcott Ct., Sacramento, CA, 94206-4056",1
5,6,Furniture Gallery,"325 Flatiron Dr., Boulder, CO, 80514-4432",0
6,7,Period Furniture,"394 Rainbow Dr., Seattle, WA, 97954-5589",0
7,8,California Classics,"816 Peach Rd., Santa Clara, CA, 96915-7754",1
8,9,M and H Casual Furniture,"3709 First Street, Clearwater, FL, 34620-2314",0
9,10,Seminole Interiors,"2400 Rocky Point Dr., Seminole, FL, 34646-4423",0


In [83]:
# 758J
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.concat_ws.html
# https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.functions.count.html
from pyspark.sql.functions import concat_ws
from pyspark.sql.functions import count
sp_customer.join(sp_order, 'customerId', 'left')\
 .select('customerId',
     'customerName',
     concat_ws(
       ', ',
       sp_customer.customerStreet,
       sp_customer.customerCity,
       sp_customer.customerState,
       sp_customer.customerZip
     ).alias('Address'),
     'orderId')\
 .groupBy('customerId', 'customerName', 'Address')\
 .agg(count('orderId')\
   .alias('Count of Orders'))\
 .orderBy('customerId').show(truncate=False)

+----------+------------------------+---------------------------------------------------+---------------+
|customerId|customerName            |Address                                            |Count of Orders|
+----------+------------------------+---------------------------------------------------+---------------+
|1         |Contemporary Casuals    |1355 S Hines Blvd, Gainesville, FL, 32601-2871     |2              |
|2         |Value Furniture         |15145 S.W. 17th St., Plano, TX, 75094-7743         |1              |
|3         |Home Furnishings        |1900 Allard Ave., Albany, NY, 12209-1125           |1              |
|4         |Eastern Furniture       |1925 Beltline Rd., Carteret, NJ, 07008-3188        |1              |
|5         |Impressions             |5585 Westcott Ct., Sacramento, CA, 94206-4056      |1              |
|6         |Furniture Gallery       |325 Flatiron Dr., Boulder, CO, 80514-4432          |0              |
|7         |Period Furniture        |394 Rainb